In [ ]:
!pip install prophet

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns

In [ ]:
BTC = pd.read_csv('./bitcoin.csv')
BTC.head()

In [ ]:
BTC.shape

In [ ]:
BTC.tail(2)

In [ ]:
BTC.info()

In [ ]:
BTC.isnull().sum()

In [ ]:
# convert the 'Date' column to datetime format
BTC['date']= pd.to_datetime(BTC['date'])

In [ ]:
BTC.shape

In [ ]:
# Renaming the columns names
BTC.rename(columns = {'date':'ds', 'price':'y'}, inplace = True)
BTC.head(2)

In [ ]:
from statsmodels.tsa.stattools import adfuller

btc_price_vol = BTC[['y', 'total_volume']]
btc_price_vol.head()


In [ ]:
result = adfuller(btc_price_vol.y.values, autolag='AIC')

print(f'ADF Statistic: {result[0]}')

print(f'p-value: {result[1]}')

for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

In [ ]:
model = Prophet(interval_width=0.8)
model.fit(BTC)

## We created data for the next two months.

In [ ]:
future = model.make_future_dataframe(periods=2, freq='M')

In [ ]:
# Let us now use this dataset to make predictions for the future using the model.

forecast = model.predict(future)

In [ ]:
# visualizing time series analysis
fig1 = model.plot(forecast)

In [ ]:
fig2 = model.plot_components(forecast)

In [ ]:
BTC['cap'] = 80000

In [ ]:
# confidence interval of 50%
model_logistic = Prophet(growth='logistic', interval_width=0.5)
# fitting the model
model_logistic.fit(BTC)

In [ ]:
# forecasting for future
future = model_logistic.make_future_dataframe(periods=2, freq='M')
# carrying capacity
future['cap'] = 15000
# predictions 
forecast_logistic = model_logistic.predict(future)

In [ ]:
fig1 = model.plot(forecast_logistic)

In [ ]:
fig2 = model.plot_components(forecast_logistic)

In [ ]:
split_date = '2022-10-24'
train_df = BTC.loc[BTC['ds'] <= split_date].copy()
test_df = BTC.loc[BTC['ds'] > split_date].copy()

In [ ]:
train_df.head(2)

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
# Setup and train model
model = Prophet()
model.fit(train_df)

In [ ]:
# Predict on training set with model
data_test_fcst = model.predict(df=test_df.reset_index())

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = model.plot(data_test_fcst, ax=ax)

In [ ]:
fig = model.plot_components(data_test_fcst)


In [ ]:
test_df.head(2)


## Plot the forecast with the actuals

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)

ax.scatter(test_df.ds, test_df['y'], color='r')

fig = model.plot(data_test_fcst, ax=ax)

# Plot the forecast with the actuals

In [ ]:
data_test_fcst.head()

In [ ]:
import datetime as dt

f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)

ax.scatter(test_df.ds, test_df['y'], color='r')

fig = model.plot(data_test_fcst, ax=ax)

# create datetime objects from string representations of dates
lower = dt.datetime(2022, 12, 23)
upper = dt.datetime(2022, 12, 31)

ax.set_xbound(lower=lower, upper=upper)



plot = plt.suptitle('Dec-2023 Forecast vs Actuals')

In [ ]:
mean_absolute_error(y_true=test_df['y'],
                   y_pred=data_test_fcst['yhat'])